In [102]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from pathlib import Path

from transformers import AutoImageProcessor, AutoModelForDepthEstimation
from ultralytics import YOLO

from matplotlib import pyplot as plt

In [103]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# MiDaS v3.1
midas = torch.hub.load("intel-isl/MiDaS", "MiDaS").to(device).eval()
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform_midas = midas_transforms.default_transform

# Depth Anything V2 (Hugging Face)
processor = AutoImageProcessor.from_pretrained("depth-anything/Depth-Anything-V2-Large-hf")
depth_anything = AutoModelForDepthEstimation.from_pretrained("depth-anything/Depth-Anything-V2-Large-hf").to(device).eval()

# YOLOv11 (entrenado por ayudante)
yolo = YOLO("yolo11s.pt")  # Reemplaza con la ruta correcta si es necesario


Using cache found in C:\Users\jtapi/.cache\torch\hub\intel-isl_MiDaS_master


Loading weights:  None


Using cache found in C:\Users\jtapi/.cache\torch\hub\facebookresearch_WSL-Images_main
Using cache found in C:\Users\jtapi/.cache\torch\hub\intel-isl_MiDaS_master


In [104]:
def estimar_profundidad_midas(img, model, transform):
    input_batch = transform(img).to(device)
    with torch.no_grad():
        prediction = model(input_batch)
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()
    return prediction.cpu().numpy()

def estimar_profundidad_depth_anything_v2(img):
    img_pil = Image.fromarray(img)
    inputs = processor(images=img_pil, return_tensors="pt").to(device)
    with torch.no_grad():
        prediction = depth_anything(**inputs).predicted_depth
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img_pil.size[::-1],
            mode="bicubic",
            align_corners=False,
        ).squeeze()
    return prediction.cpu().numpy()

def detectar_y_medir_distancia(img, depth_map, model, conf=0.6):
    results = model.predict(img, conf=conf, verbose=False)[0]
    detecciones = []
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cls = int(box.cls[0])
        confianza = float(box.conf[0])
        label = model.names[cls]
        roi = depth_map[y1:y2, x1:x2]
        distancia = np.median(roi) if roi.size else 0
        detecciones.append({
            "bbox": (x1, y1, x2, y2),
            "label": label,
            "confianza": confianza,
            "distancia": round(float(distancia), 2)
        })
    return detecciones

def dibujar_detecciones(img, detecciones):
    salida = img.copy()
    for det in detecciones:
        x1, y1, x2, y2 = det["bbox"]
        texto = f'{det["label"]} {det["confianza"]:.2f} | {det["distancia"]} m'
        cv2.rectangle(salida, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(salida, texto, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)
    return salida

def resize_img(img, height=200):
    aspect_ratio = img.shape[1] / img.shape[0]
    return cv2.resize(img, (int(height * aspect_ratio), height))

def guardar_mapa_profundidad(depth_map, ruta_salida):
    norm = (depth_map - np.min(depth_map)) / (np.max(depth_map) - np.min(depth_map) + 1e-8)
    plt.imsave(ruta_salida, norm, cmap="inferno")


In [105]:
# Rutas
frames_dir = Path("../frames")
out_root = Path("../outputs")
out_midas = out_root / "modelo_midas"
out_da = out_root / "modelo_depth_anything"
out_yolo_midas = out_root / "modelo_midas_yolo"
out_yolo_da = out_root / "modelo_depth_anything_yolo"
out_summary = Path("../summarize")

# Crear carpetas
for p in [out_midas, out_da, out_yolo_midas, out_yolo_da, out_summary]:
    p.mkdir(parents=True, exist_ok=True)

# Leer 5 imágenes
frame_paths = sorted([p for p in frames_dir.glob("*.jpg")])[:5]
frames = [cv2.cvtColor(cv2.imread(str(p)), cv2.COLOR_BGR2RGB) for p in frame_paths]
frame_names = [p.stem for p in frame_paths]

# Procesar y guardar todo
resumen_filas = []



# Crear resumen visual tipo plot (1 fila x 5 columnas)
for frame, name in zip(frames, frame_names):
    # Estimar mapas
    depth_midas = estimar_profundidad_midas(frame, midas, transform_midas)
    depth_da = estimar_profundidad_depth_anything_v2(frame)

    # Guardar mapas
    guardar_mapa_profundidad(depth_midas, out_midas / f"{name}_midas.png")
    guardar_mapa_profundidad(depth_da, out_da / f"{name}_da.png")

    # YOLO detección
    det_midas = detectar_y_medir_distancia(frame, depth_midas, yolo)
    det_da = detectar_y_medir_distancia(frame, depth_da, yolo)

    # Dibujar y guardar detecciones
    img_yolo_midas = dibujar_detecciones(frame, det_midas)
    img_yolo_da = dibujar_detecciones(frame, det_da)
    cv2.imwrite(str(out_yolo_midas / f"{name}_midas_yolo.png"), cv2.cvtColor(img_yolo_midas, cv2.COLOR_RGB2BGR))
    cv2.imwrite(str(out_yolo_da / f"{name}_da_yolo.png"), cv2.cvtColor(img_yolo_da, cv2.COLOR_RGB2BGR))

    



In [ ]:

# Re-ensamblar los componentes para el plot completo
fig, axs = plt.subplots(5, 5, figsize=(20, 16))
column_titles = ["Original", "Prof. MiDaS", "Prof. Depth Anything", "YOLO + MiDaS", "YOLO + Depth Anything"]

for row_idx, (frame, name) in enumerate(zip(frames, frame_names)):
    # Cargar imágenes procesadas
    img_orig = frame
    img_midas = cv2.cvtColor(cv2.imread(str(out_midas / f"{name}_midas.png")), cv2.COLOR_BGR2RGB)
    img_da = cv2.cvtColor(cv2.imread(str(out_da / f"{name}_da.png")), cv2.COLOR_BGR2RGB)
    img_yolo_midas = cv2.cvtColor(cv2.imread(str(out_yolo_midas / f"{name}_midas_yolo.png")), cv2.COLOR_BGR2RGB)
    img_yolo_da = cv2.cvtColor(cv2.imread(str(out_yolo_da / f"{name}_da_yolo.png")), cv2.COLOR_BGR2RGB)

    # Agregar a cada celda del plot
    row_images = [img_orig, img_midas, img_da, img_yolo_midas, img_yolo_da]
    for col_idx, img in enumerate(row_images):
        axs[row_idx, col_idx].imshow(img)
        axs[row_idx, col_idx].axis("off")
        if row_idx == 0:
            axs[row_idx, col_idx].set_title(column_titles[col_idx], fontsize=14)

# Ajustar diseño y guardar
plt.tight_layout()
summary_final_path = out_summary / "comparativa_summary_total.png"
plt.savefig(summary_final_path)
plt.close(fig)

print(f"✅ Imagen resumen final 5x5 guardada como: {summary_final_path}")


✅ Imagen resumen final 5x5 guardada como: ..\summarize\comparativa_summary_total.png
